In [1]:
import pandas as pd
import os
import sys
import win32cred
import requests
from datetime import datetime
import scipy.stats as stats
import matplotlib.pyplot as plt
from eodhd import APIClient

from ticker_loader import *

# Loading the composition of the S&P 500 index from a csv file

In [2]:
spy_df, all_stocks = load_SPY_components()
spy_df

,Date,Tickers,Number of Stocks
0,1996-01-01,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD...",487
1,1996-02-01,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD...",487
2,1996-03-01,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD...",487
3,1996-04-01,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD...",487
4,1996-05-01,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD...",487
...,...,...,...
311,2022-06-01,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,...",504
312,2022-09-01,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,...",503
313,2022-10-01,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,...",503
314,2022-11-01,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACGL,ACN,ADBE...",503


In [3]:
print(f"All {len(all_stocks)} stocks : ", all_stocks)

All 1136 stocks :  ['KSS', 'DELL', 'RSHCQ', 'ISRG', 'WDC', 'MMC', 'CHTR', 'URBN', 'KATE', 'AZO', 'DXCM', 'CHRW', 'BBT', 'EA', 'SRE', 'SJM', 'T', 'MO', 'MDT', 'BOL', 'ADP', 'DHR', 'NYN', 'RAL', 'TSN', 'SGID', 'STE', 'GNW', 'CMB', 'AFL', 'NAE', 'MTG', 'SB', 'VTSS', 'CR', 'HSP', 'ADI', 'PVH', 'NEE', 'BDK', 'JPM', 'APD', 'GLK', 'FWLT', 'HAL', 'AABA', 'D', 'GS', 'PAC', 'GL', 'RTX', 'WST', 'TEK', 'IRM', 'HES', 'BRCM', 'TT', 'MOH', 'CMX', 'PRU', 'WY', 'URI', 'MAS', 'MWI', 'MRK', 'HFS', 'FIS', 'UAWGQ', 'ABT', 'WCOEQ', 'GDW', 'ESRX', 'TER', 'CCB', 'VIAB', 'H', 'LMT', 'HCBK', 'JAVA', 'ODFL', 'BIO', 'UAL', 'ENPH', 'APA', 'HLT', 'AAL', 'FRC', 'ECH', 'KM', 'ACS', 'PSA', 'UPC', 'BBY', 'UNM', 'SLG', 'HOT', 'PNR', 'HOLX', 'ADM', 'ARNC', 'EFX', 'SO', 'TEL', 'DGN', 'MEL', 'MIL', 'JOS', 'VZ', 'TRIP', 'FJ', 'SE', 'CMVT', 'BKR', 'TXU', 'ASND', 'NLSN', 'PHB', 'ADT', 'UCL', 'BSX', 'AVP', 'IPG', 'DJ', 'KMG', 'TCOMA', 'LVS', 'HRB', 'I', 'CVH', 'GME', 'PBY', 'XOM', 'ETFC', 'MLM', 'FMY', 'MYL', 'SBAC', 'PEG', 'L

# 0. Connecting to the API

In [2]:
def get_credentials(target_name):
    creds = win32cred.CredRead(target_name, win32cred.CRED_TYPE_GENERIC)
    credential_blob = creds["CredentialBlob"]
    password = credential_blob.decode('utf-16-le').strip()
    return password

creds = get_credentials("eodhd.com")
api_client = APIClient(creds)

# 1. Get a list of exchanges available in this API

In [3]:
df = api_client.get_exchanges()
pd.set_option('display.max_columns', None)
df

,Name,Code,OperatingMIC,Country,Currency,CountryISO2,CountryISO3
0,USA Stocks,US,"XNAS, XNYS, OTCM",USA,USD,US,USA
1,London Exchange,LSE,XLON,UK,GBP,GB,GBR
2,Toronto Exchange,TO,XTSE,Canada,CAD,CA,CAN
3,TSX Venture Exchange,V,XTSX,Canada,CAD,CA,CAN
4,NEO Exchange,NEO,NEOE,Canada,CAD,CA,CAN
...,...,...,...,...,...,...,...
73,Money Market Virtual Exchange,MONEY,None,Unknown,Unknown,,
74,Europe Fund Virtual Exchange,EUFUND,None,Unknown,EUR,,
75,Istanbul Stock Exchange,IS,XIST,Turkey,TRY,TR,TUR
76,FOREX,FOREX,CDSL,Unknown,Unknown,,


# 2. Get historical price data of stocks

Testing the API with NVDA (NVIDIA Corporation) stock as the prices look very unusual in this time frame.

In [9]:
resp = api_client.get_eod_historical_stock_market_data(symbol = 'NVDA', period='d', from_date = '2024-06-06', to_date = '2024-06-12', order='a')
df = pd.DataFrame(resp)
df

,date,open,high,low,close,adjusted_close,volume
0,2024-06-06,1240.48000000,1255.87000000,1183.20000000,1209.98000000,120.97770000,664696163
1,2024-06-07,1197.70000000,1216.91700000,1180.22000000,1208.88000000,120.86770000,412385776
2,2024-06-10,120.37000000,123.10000000,117.01000000,121.79000000,121.76960000,314162688
3,2024-06-11,121.77000000,122.87000000,118.74000000,120.91000000,120.89970000,222551203
4,2024-06-12,123.06000000,126.88000000,122.57000000,125.20000000,125.18930000,299595000


# 3. Get Financial Statements of a ticker

In [7]:
resp = api_client.get_fundamentals_data(ticker = "NVDA")
resp.keys()

dict_keys(['General', 'Highlights', 'Valuation', 'SharesStats', 'Technicals', 'SplitsDividends', 'AnalystRatings', 'Holders', 'InsiderTransactions', 'ESGScores', 'outstandingShares', 'Earnings', 'Financials'])

In [8]:
resp["General"]

{'Code': 'NVDA',
 'Type': 'Common Stock',
 'Name': 'NVIDIA Corporation',
 'Exchange': 'NASDAQ',
 'CurrencyCode': 'USD',
 'CurrencyName': 'US Dollar',
 'CurrencySymbol': '$',
 'CountryName': 'USA',
 'CountryISO': 'US',
 'OpenFigi': 'BBG000BBJQV0',
 'ISIN': 'US67066G1040',
 'LEI': '549300S4KLFTLO7GSQ80',
 'PrimaryTicker': 'NVDA.US',
 'CUSIP': '67066G104',
 'CIK': '1045810',
 'EmployerIdNumber': '94-3177549',
 'FiscalYearEnd': 'January',
 'IPODate': '1999-01-22',
 'InternationalDomestic': 'Domestic',
 'Sector': 'Technology',
 'Industry': 'Semiconductors',
 'GicSector': 'Information Technology',
 'GicGroup': 'Semiconductors & Semiconductor Equipment',
 'GicIndustry': 'Semiconductors & Semiconductor Equipment',
 'GicSubIndustry': 'Semiconductors',
 'HomeCategory': 'Domestic',
 'IsDelisted': False,
 'Description': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce 

In [9]:
resp["Financials"].keys()

dict_keys(['Balance_Sheet', 'Cash_Flow', 'Income_Statement'])

In [10]:
resp["Financials"]["Balance_Sheet"]["quarterly"].keys()

dict_keys(['2024-07-31', '2024-04-30', '2024-01-31', '2023-10-31', '2023-07-31', '2023-04-30', '2023-01-31', '2022-10-31', '2022-07-31', '2022-04-30', '2022-01-31', '2021-10-31', '2021-07-31', '2021-04-30', '2021-01-31', '2020-10-31', '2020-07-31', '2020-04-30', '2020-01-31', '2019-10-31', '2019-07-31', '2019-04-30', '2019-01-31', '2018-10-31', '2018-07-31', '2018-04-30', '2018-01-31', '2017-10-31', '2017-07-31', '2017-04-30', '2017-01-31', '2016-10-31', '2016-07-31', '2016-04-30', '2016-01-31', '2015-10-31', '2015-07-31', '2015-04-30', '2015-01-31', '2014-10-31', '2014-07-31', '2014-04-30', '2014-01-31', '2013-10-31', '2013-07-31', '2013-04-30', '2013-01-31', '2012-10-31', '2012-07-31', '2012-04-30', '2012-01-31', '2011-10-31', '2011-07-31', '2011-04-30', '2011-01-31', '2010-10-31', '2010-07-31', '2010-04-30', '2010-01-31', '2009-10-31', '2009-07-31', '2009-04-30', '2009-01-31', '2008-10-31', '2008-07-31', '2008-04-30', '2008-01-31', '2007-10-31', '2007-07-31', '2007-04-30', '2007-01-

In [11]:
resp["Financials"]["Balance_Sheet"]["quarterly"]["2024-07-31"]

{'date': '2024-07-31',
 'filing_date': '2024-08-28',
 'currency_symbol': 'USD',
 'totalAssets': '85227000000.00',
 'intangibleAssets': '952000000.00',
 'earningAssets': None,
 'otherCurrentAssets': '4026000000.00',
 'totalLiab': '27070000000.00',
 'totalStockholderEquity': '58157000000.00',
 'deferredLongTermLiab': None,
 'otherCurrentLiab': '9091000000.00',
 'commonStock': '25000000.00',
 'capitalStock': '25000000.00',
 'retainedEarnings': '45961000000.00',
 'otherLiab': None,
 'goodWill': '4622000000.00',
 'otherAssets': None,
 'cash': '8563000000.00',
 'cashAndEquivalents': None,
 'totalCurrentLiabilities': '13969000000.00',
 'currentDeferredRevenue': '948000000.00',
 'netDebt': '1452000000.00',
 'shortTermDebt': '250000000.00',
 'shortLongTermDebt': None,
 'shortLongTermDebtTotal': '10015000000.00',
 'otherStockholderEquity': '12115000000.00',
 'propertyPlantEquipment': None,
 'totalCurrentAssets': '59633000000.00',
 'longTermInvestments': '1819000000.00',
 'netTangibleAssets': Non

In [12]:
resp["Financials"]["Cash_Flow"]["quarterly"]["2024-07-31"]

{'date': '2024-07-31',
 'filing_date': '2024-08-28',
 'currency_symbol': 'USD',
 'investments': '-3184000000.00',
 'changeToLiabilities': None,
 'totalCashflowsFromInvestingActivities': None,
 'netBorrowings': None,
 'totalCashFromFinancingActivities': '-10320000000.00',
 'changeToOperatingActivities': None,
 'netIncome': '16599000000.00',
 'changeInCash': '984000000.00',
 'beginPeriodCashFlow': '7587000000.00',
 'endPeriodCashFlow': '8571000000.00',
 'totalCashFromOperatingActivities': '14488000000.00',
 'issuanceOfCapitalStock': None,
 'depreciation': '433000000.00',
 'otherCashflowsFromInvestingActivities': None,
 'dividendsPaid': '246000000.00',
 'changeToInventory': '-803000000.00',
 'changeToAccountReceivables': '-1767000000.00',
 'salePurchaseOfStock': '-7158000000.00',
 'otherCashflowsFromFinancingActivities': '-1666000000.00',
 'changeToNetincome': None,
 'capitalExpenditures': '977000000',
 'changeReceivables': None,
 'cashFlowsOtherOperating': None,
 'exchangeRateChanges': N

In [13]:
resp["Financials"]["Income_Statement"]["quarterly"]["2024-07-31"]

{'date': '2024-07-31',
 'filing_date': '2024-08-28',
 'currency_symbol': 'USD',
 'researchDevelopment': '3090000000.00',
 'effectOfAccountingCharges': None,
 'incomeBeforeTax': '19214000000.00',
 'minorityInterest': None,
 'netIncome': '16599000000.00',
 'sellingGeneralAdministrative': '842000000.00',
 'sellingAndMarketingExpenses': None,
 'grossProfit': '22574000000.00',
 'reconciledDepreciation': '433000000.00',
 'ebit': '19275000000.00',
 'ebitda': '19708000000.00',
 'depreciationAndAmortization': '433000000.00',
 'nonOperatingIncomeNetOther': None,
 'operatingIncome': '18642000000.00',
 'otherOperatingExpenses': '11398000000.00',
 'interestExpense': '61000000.00',
 'taxProvision': '2615000000.00',
 'interestIncome': '444000000.00',
 'netInterestIncome': '383000000.00',
 'extraordinaryItems': None,
 'nonRecurring': None,
 'otherItems': None,
 'incomeTaxExpense': '2615000000.00',
 'totalRevenue': '30040000000.00',
 'totalOperatingExpenses': '3932000000.00',
 'costOfRevenue': '7466000

In [14]:
resp["Earnings"].keys()

dict_keys(['History', 'Trend', 'Annual'])

In [15]:
resp["Earnings"]["History"]["2023-07-31"]

{'reportDate': '2023-08-23',
 'date': '2023-07-31',
 'beforeAfterMarket': 'AfterMarket',
 'currency': 'USD',
 'epsActual': 0.27,
 'epsEstimate': 0.207,
 'epsDifference': 0.063,
 'surprisePercent': 30.4348}

In [16]:
resp["Earnings"]["Trend"]["2024-07-31"]

{'date': '2024-07-31',
 'period': '0q',
 'growth': '1.5600',
 'earningsEstimateAvg': '0.6400',
 'earningsEstimateLow': '0.6000',
 'earningsEstimateHigh': '0.7100',
 'earningsEstimateYearAgoEps': '0.2500',
 'earningsEstimateNumberOfAnalysts': '39.0000',
 'earningsEstimateGrowth': '1.5600',
 'revenueEstimateAvg': '25675700000.00',
 'revenueEstimateLow': '23992800000.00',
 'revenueEstimateHigh': '27245700000.00',
 'revenueEstimateYearAgoEps': None,
 'revenueEstimateNumberOfAnalysts': '40.00',
 'revenueEstimateGrowth': None,
 'epsTrendCurrent': '0.6400',
 'epsTrend7daysAgo': '0.5700',
 'epsTrend30daysAgo': '0.6300',
 'epsTrend60daysAgo': '0.6300',
 'epsTrend90daysAgo': '5.8600',
 'epsRevisionsUpLast7days': '3.0000',
 'epsRevisionsUpLast30days': '8.0000',
 'epsRevisionsDownLast7days': None,
 'epsRevisionsDownLast30days': '0.0000'}

In [17]:
resp["Earnings"]["Annual"]["2024-07-31"]

{'date': '2024-07-31', 'epsActual': 1.292}

In [18]:
resp.keys()

dict_keys(['General', 'Highlights', 'Valuation', 'SharesStats', 'Technicals', 'SplitsDividends', 'AnalystRatings', 'Holders', 'InsiderTransactions', 'ESGScores', 'outstandingShares', 'Earnings', 'Financials'])

In [19]:
resp["Highlights"]

{'MarketCapitalization': 3064287461376,
 'MarketCapitalizationMln': 3064287.4614,
 'EBITDA': 61184000000,
 'PERatio': 58.3738,
 'PEGRatio': 0.9932,
 'WallStreetTargetPrice': 148.13,
 'BookValue': 2.368,
 'DividendShare': 0.022,
 'DividendYield': 0.0003,
 'EarningsShare': 2.14,
 'EPSEstimateCurrentYear': 12.41,
 'EPSEstimateNextYear': 22.1,
 'EPSEstimateNextQuarter': 0,
 'EPSEstimateCurrentQuarter': 0.64,
 'MostRecentQuarter': '2024-07-31',
 'ProfitMargin': 0.5504,
 'OperatingMarginTTM': 0.6206,
 'ReturnOnAssetsTTM': 0.5526,
 'ReturnOnEquityTTM': 1.2377,
 'RevenueTTM': 96307003392,
 'RevenuePerShareTTM': 3.91,
 'QuarterlyRevenueGrowthYOY': 1.224,
 'GrossProfitTTM': 44301000000,
 'DilutedEpsTTM': 2.14,
 'QuarterlyEarningsGrowthYOY': 1.68}

In [20]:
resp["Technicals"]

{'Beta': 1.669,
 '52WeekHigh': 140.7479,
 '52WeekLow': 39.2182,
 '50DayMA': 116.136,
 '200DayMA': 96.1379,
 'SharesShort': 275330404,
 'SharesShortPriorMonth': 260873913,
 'ShortRatio': 0.8,
 'ShortPercent': 0.0117}

In [21]:
resp["Valuation"]

{'TrailingPE': 58.3738,
 'ForwardPE': 32.8947,
 'PriceSalesTTM': 31.8179,
 'PriceBookMRQ': 52.6899,
 'EnterpriseValue': 3039502600000,
 'EnterpriseValueRevenue': 31.5606,
 'EnterpriseValueEbitda': 48.266}

# Get Macro Economic Data

In [22]:
indicator_extended = api.get_macro_indicators_data (country = "USA")
indicator_m = pd.DataFrame(indicator_extended) 
print(indicator_m)

NameError: name 'api' is not defined

: 

In [ ]:
news_extended = api.financial_news (from_date = "2020-12-10", to_date = "2021-04-10", t = "financial results", offset = "200", limit = "100")
fn_m = pd.DataFrame(news_extended) 
print(fn_m)

                         date  \
0   2021-03-31T21:00:00+00:00   
1   2021-03-31T20:10:00+00:00   
2   2021-03-31T20:10:00+00:00   
3   2021-03-31T19:05:00+00:00   
4   2021-03-31T19:05:00+00:00   
..                        ...   
95  2021-03-24T12:00:00+00:00   
96  2021-03-24T04:00:30+00:00   
97  2021-03-23T21:00:00+00:00   
98  2021-03-23T20:05:00+00:00   
99  2021-03-23T20:05:00+00:00   

                                                title  \
0   Brown &amp; Brown, Inc. Announces 2021 First-Q...   
1   Verint, Cognyte's Former Parent, Announces Fou...   
2   Verint, Cognyte's Former Parent, Announces Fou...   
3   Net Element Reports Full Year 2020 Financial R...   
4   Net Element Reports Full Year 2020 Financial R...   
..                                                ...   
95  AYRO to Report Year-End 2020 Financial Results...   
96    GameStop (GME) Q4 2020 Earnings Call Transcript   
97                            Labcorp Business Update   
98  Golden Nugget Online Gaming, 